In [ ]:

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
from tqdm import tqdm

In [ ]:
# import pandas as pd

# # Load the data
# data = pd.read_csv('/kaggle/working/GT.csv')

# # Remove all rows after the 3000th row
# data = data.iloc[:2000]

# # Save the updated data to a new CSV file
# data.to_csv('/kaggle/working/GT.csv', index=False)

In [ ]:
data = pd.read_csv('/kaggle/input/isicgt25k/ISIC_2019_Training_GroundTruth.csv')
data.head()

In [ ]:
store_list = []
image_height = 224
image_width = 224
for i in tqdm(range(data.shape[0])):
    path = '/kaggle/input/isic2019/ISIC_2019_Training_Input/' + data['image'][i] + '.jpg'
    image_check = image.load_img(path, target_size=(image_height, image_width))
    image_check = image.img_to_array(image_check)
    # scaling the images
    image_check = image_check/255
    store_list.append(image_check)

In [ ]:
x = np.array(store_list)
y = data.drop(columns=['image'])
Y = y.to_numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, Y, random_state=42, test_size=0.20)


In [ ]:
#VGGNET
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=x_train[0].shape))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2), strides=(2, 2)))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2), strides=(2, 2)))

# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2), strides=(2, 2)))

# Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2), strides=(2, 2)))

# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2), strides=(2, 2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model(Dropout(0.5))
model.add(Dense(9, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
es_callback = keras.callbacks.EarlyStopping(patience=3)
history = model.fit(x_train, y_train, epochs=15, validation_split=0.3, callbacks=[es_callback]

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Make predictions on the test set
y_pred = model.predict(x_test)

# Print the actual and predicted data side by side
for i in range(len(y_test)):
    print('Actual:', y_test[i], 'Predicted:', y_pred[i])

# Plot the accuracy and loss graphs
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()